# Make a tree

### Initialisation

In [ ]:
import os, json
import numpy as np

testydata: str = "./../../testymolo/media/data/"

#tree1:str = ""
#Organism:list = []
#forest  = np.zeros((8,8))

with open(os.path.join(testydata, "Organism.json")) as handle:
    Organism:list = json.loads(handle.read())
    print(Organism[0])
    

In [ ]:
"""
### ADD
# HIV
#Organism.append({
#    'TaxId':"11676",
#    'Name':"Human immunodeficiency virus 1",
#    'abr':"HIV-1",
#    'phylogeny':"Viruses; Riboviria; Pararnavirae; Artverviricota; Revtraviricetes; Ortervirales; Retroviridae; Orthoretrovirinae; Lentivirus"
#})
#Organism.append({
#    'TaxId':"11709",
#    'Name':"Human immunodeficiency virus 2",
#    'abr':"HIV-2",
#    'phylogeny':"Viruses; Riboviria; Pararnavirae; Artverviricota; Revtraviricetes; Ortervirales; Retroviridae; Orthoretrovirinae; Lentivirus"
#})
# Rift Valley fever virus
Organism.append({
    'TaxId':"11588",
    'Name':"Rift Valley fever virus",
    'abr':"RVFV",
    'phylogeny':"Viruses; Riboviria; Orthornavirae; Negarnaviricota; Polyploviricotina; Ellioviricetes; Bunyavirales; Phenuiviridae; Phlebovirus; Phlebovirus riftense"
})
# Machupo
Organism.append({
    'TaxId':"3052317",
    'Name':"Mammarenavirus machupoense",
    'abr':"Machupo",
    'phylogeny':"Viruses; Riboviria; Orthornavirae; Negarnaviricota; Polyploviricotina; Ellioviricetes; Bunyavirales; Arenaviridae; Mammarenavirus"
})
# zika
Organism.append({
    'TaxId':"64320",
    'Name':"Zika virus",
    'abr':"zika",
    'phylogeny':"Viruses; Riboviria; Orthornavirae; Kitrinoviricota; Flasuviricetes; Amarillovirales; Flaviviridae; Orthoflavivirus; Orthoflavivirus zikaense"
})
# SARS-CoV-2
Organism.append({
    'TaxId':"2697049",
    'Name':"Severe acute respiratory syndrome coronavirus 2",
    'abr':"SARS-CoV-2",
    'phylogeny':"Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Betacoronavirus; Sarbecovirus; Severe acute respiratory syndrome-related coronavirus"
})
# mers
Organism.append({
    'TaxId':"1335626",
    'Name':"Middle East respiratory syndrome-related coronavirus",
    'abr':"MERS",
    'phylogeny':"Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Betacoronavirus; Merbecovirus"
})
"""

### Functions

In [ ]:

def gimme(A:list, B:list) -> int:
    # give count of differences between to phylogenic paths A and B
    c:int = 0
    a,b = 0,0
    while(a < len(A) and b < len(B)):
        if(A[a] != B[b]):
            c += 1
        a += 1
        b += 1
    return c
            
def cupid(people:list[tuple]) -> list:
    # people: list of ids
    # found the closest couple within the list
    # recursive: new people(list) n-1, couple formed
    if(len(people) <= 2):
        #end 
        #forme the last couple -> root
        return [people[0], people[1]]
    else:
        #found couple
        N:int = len(people)
        # -> calc matrix of differences of remaining people
        NEO:np.array = neo(np.zeros((N,N)))
        # -> -> forme new couple of the lowest score 
        a,b,z = morpheus(NEO)
        # -> -> -> new people with couple formed 
        remaining_people:list = []
        for i in range(len(people)):
            if not (i == a or i == b):
                remaining_people.append(people[i])
        remaining_people.append([people[a], people[b]])
        print(people[a], people[b])
        return cupid(remaining_people)

def neo(forest:np.array) -> np.array:
    for i in range(len(forest)):
        for j in range(len(forest)):
            if(i == j or forest[i][j] != 0):
                # ignore diagonal 
                # if value already set (!=0)
                continue
            A = Organism[i]['phylogeny'].split(';')
            A = [item.strip() for item in A]
            B = Organism[j]['phylogeny'].split(';')
            B = [item.strip() for item in B]
            forest[i][j] = gimme(A,B)
            forest[j][i] = gimme(A,B)
    #print(forest)
    return forest

def commun_grounds(A:str, B:str) -> str:
    A:list = [a.strip() for a in A.split(';')]
    B:list = [b.strip() for b in B.split(';')]
    AB:list = []
    i,j = 0,0
    while(A[i] == B[j]):
        AB.append(A[i])
        i += 1 
        j += 1
    return '; '.join(AB)

def morpheus(neo:np.array) -> tuple :
    # transfrom neo into smith (matrix diagonal)
    smith = agent_smith(neo)
    lowest = lowerdeck(smith)
    pills:tuple = firstOccur(smith, lowest)
    return pills

def agent_smith(neo:np.array) -> list:
    N = len(neo)
    smith = []
    for k in range(1,N):
        smith.append([])
        for i in range(k):
            smith[k-1].append(neo[k][i])
    return smith

def lowerdeck(smith:list) -> int:
    lowest = 100
    for i in range(len(smith)):
        for j in range(len(smith[i])):
            lowest = min(smith[i][j], lowest)
    return lowest

def firstOccur(Matrix, value) -> tuple:
    for i in range(len(Matrix)):
        for j in range(len(Matrix[i])):
            if(Matrix[i][j] == value):
                return (i, j, value)

def rewind(people:list, tree:str="") -> str:
    if(len(people) < 1):
        #if int
        if isinstance(people[0], int):
            work:str = Organism[people[0]]['abr'].replace(' ', '_')
            return '(' + tree + ',' + work + ')'
        #if list
        elif isinstance(people[0], list):
            return rewind(people(0), tree) + ',' + tree
    else:
        txt:str = "("
        for k in people:
            if isinstance(k, int):
                work:str = Organism[k]['abr'].replace(' ', '_')
                txt += work
            elif isinstance(k, list):
                txt += rewind(k, tree)
            txt += ','
        tree = txt[:-1] + ')' + tree            
        
    return tree


In [ ]:
import operator
Organism_sorted:list = sorted(Organism, key=operator.itemgetter('phylogeny'))
print('\n'.join( [org['phylogeny'] for org in Organism_sorted] ))

In [ ]:
import operator
Organism_sorted:list = sorted(Organism, key=operator.itemgetter('phylogeny'))
for org in Organism_sorted:
    phylopath:list = org['phylogeny'].split(';')
    phylopath = [word.strip() for word in phylopath]
    org['phylogeny'] = list(phylopath)


print(len(Organism_sorted))
for org in Organism_sorted:
    print(org['phylogeny'])



In [ ]:
"""def is_abr_save(abr:str) -> bool:
    unsafes:list = ['.', ',', '[', ']', '/', ' ', '(', ')']
    if abr is None:
        return False
    for char in abr :
        if char in unsafes:
            return False
    return True

def clean_abr(abr:str) -> str:
    if abr is None:
        return "None"
    abr = abr.replace('[', '_')
    abr = abr.replace(']', '_')
    abr = abr.replace('/', '_')
    abr = abr.replace('-', '_')
    abr = abr.replace('.', '_')
    abr = abr.replace(',', '_')
    abr = abr.replace('(', '_')
    abr = abr.replace(')', '_')
    abr = abr.replace(' ', '_')
    return abr.strip()


for org in Organism_sorted:
    abr = org['abr']
    if not is_abr_save(abr):
        abr = clean_abr(abr)
    print(abr)
    
"""

In [ ]:

def Phedre(tree:list, ni:int = 0) -> list:
    # tree:list => idk
    # ni:int => navigation index

    # Current number 
    N:int = len(tree)

    # who is what ? 
    #print([ot for ot in range(len(tree))], ' | ', ni)

    # collect keywords 
    keywords:list = []
    finishers:list = []
    for org in tree: 
        if ni < len(org['phylogeny']):
            keyword:str = org['phylogeny'][ni]
        else:
            keyword:str = None
            finishers.append(org)
        keywords.append(keyword)
    #print("keywords", keywords)

    # Count
    # auto-sort: alphabetic order
    K:dict = {}
    k:int = -1
    for word in keywords:
        k += 1
        if not word is None:
            if word in K:
                K[word].append(k)
            else:
                K[word] = [k]
    #print("K", K)
    

    this_node:list = []

    # managing finishers
    if len(finishers) > 0:
        this_node.append([ org for org in finishers ])
    
    # recursive
    for word in K:
        new_clade:list = [ tree[tree_i] for tree_i in K[word] ]

        this_node.append(Phedre(new_clade, ni+1))
    
    #print("persue with: ", subtree)
    return this_node



In [ ]:

britannicus = Phedre(Organism_sorted)
print(britannicus)

In [ ]:
def is_abr_save(abr:str) -> bool:
    unsafes:list = ['.', ',', '/', ' ', '(', ')']
    if abr is None:
        return False
    for char in abr :
        if char in unsafes:
            return False
    return True

def clean_abr(abr:str) -> str:
    if abr is None:
        return "None"
    abr = abr.replace('/', '_')
    #abr = abr.replace('-', '_')
    abr = abr.replace('.', '_')
    abr = abr.replace(',', '_')
    abr = abr.replace('(', '_')
    abr = abr.replace(')', '_')
    abr = abr.replace(' ', '_')
    return abr.strip()

def Racine(tree:list, ni:int = 1, di:int = 0, rooted:bool = False) -> str:
    #recursive
    #ni:int :: level index -> branch length

    if not len(tree) > 1:  # only one fork
        item = tree[0]

        if isinstance(item, list):  # another node
            if rooted:  # if inside tree
                #ni += 1  # expand branch level
                #print(f"Racine(node, ni:{ni}, rooted)")
                return Racine(item, ni+1, di+1, rooted=True)  # continue further, ignore level, increase the branch length
            else:
                #print('.'*di + f"Racine(node, 1, un-rooted)")
                return Racine(item, 1, 0, False)  # continue further, ignore level
            
        if isinstance(item, dict):  # another organism dict
            #print('.'*di + f"{item['abr']}:{ni}")
            abr = item['abr']
            if not is_abr_save(abr):
                abr = clean_abr(abr)
            #print(abr)            
            return f"{str(abr)}:{str(ni)}"

    else:  # binary node (hope ^^)
        result:list = []
        for item in tree:
        
            if isinstance(item, list):  # another node
                #print('.'*di + f"Racine(node, 1, rooted)")
                result.append(Racine(item, 1, di+1, rooted=True))
                    
            elif isinstance(item, dict):
                #print('.'*di + f"{item['abr']}:{ni}")
                abr = item['abr']
                if not is_abr_save(abr):
                    abr = clean_abr(abr)
                #print(abr)
                result.append(f"{str(abr)}:{str(ni)}")

        if len(result) > 1:
            if rooted:
                return '('+ ','.join(result) +'):1'
            else:
                return '('+ ','.join(result) +');'
            
    if rooted:
        return result
    else:
        return result+';'
    

In [ ]:
arbre_test_1 = [[[{'abr':'A'},{'abr':'B'}],{'abr':'C'}],[[{'abr':'D'},{'abr':'E'}],{'abr':'F'}]]
arbre_test_2 = [[{'abr':'A'},{'abr':'B'}],[[{'abr':'C'}]]]
arbre_test_3 = [[{'abr':'A'},{'abr':'B'}],[{'abr':'C'}]]
arbre_test_4 = [[[[{'abr':'A'},{'abr':'B'}],[{'abr':'C'}]]]]



In [ ]:
import newick

tree = Racine(arbre_test_1)
print(tree)
fig = newick.loads(tree)[0]
print(fig.ascii_art())

In [ ]:
import newick

print(britannicus)
tree = Racine(britannicus)
print(tree)
fig = newick.loads(tree)[0]
print(fig.ascii_art())

In [10]:
import json, os

class Fangorn:
    testydata: str = "./../../../testymolo/media/data/"
    Organism:list = json.load(open(os.path.join(testydata,'Organism.temp.json')))

    def __init__(self, *args, **kwargs):
        self.taxo = {}
        if 'All' in args:
            for item in Fangorn.Organism:
                self.add_taxo(item['phylogeny'])
            pass
        pass

    def str2list(taxo:str) -> list:
        return list([t.strip() for t in taxo.split(';')])
    
    def list2dict(taxo:list) -> dict:
        if len(taxo) >1:
            # continue
            return { taxo[0] : Fangorn.list2dict(taxo[1:]) }
        else :
            # stop
            return { taxo[0] }

    def add_taxo(self, new_taxo:str):
        new_taxo = Fangorn.str2list(new_taxo)


        if len(self.taxo) <1:
            self.taxopath = Fangorn.list2dict(new_taxo)

        else:
            found:bool = False
            n:int = len(new_taxo)
            i:int = 0
            k:int = len(self.taxopath)
            while i < n:
                if found:
                    # completion
                    self

                    
                    and (new_taxo[i] in self.taxopath[i]):
                    found = True
                    i+=1
                if 
                else:
                    self.taxopath[i].append()

        pass


In [11]:
tree = Fangorn('All')
print(tree.taxo)

In [13]:
taxo = "Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Gammacoronavirus; Igacovirus; Avian coronavirus"
taxo = Fangorn.list2dict(Fangorn.str2list(taxo))
print(taxo)

{'Viruses': {'Riboviria': {'Orthornavirae': {'Pisuviricota': {'Pisoniviricetes': {'Nidovirales': {'Cornidovirineae': {'Coronaviridae': {'Orthocoronavirinae': {'Gammacoronavirus': {'Igacovirus': {'Avian coronavirus'}}}}}}}}}}}}


In [16]:
# plus one
t1 = "Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Betacoronavirus; Embecovirus; Betacoronavirus 1"
t1 = Fangorn.list2dict(Fangorn.str2list(t1))
print(t1)

{'Viruses': {'Riboviria': {'Orthornavirae': {'Pisuviricota': {'Pisoniviricetes': {'Nidovirales': {'Cornidovirineae': {'Coronaviridae': {'Orthocoronavirinae': {'Betacoronavirus': {'Embecovirus': {'Betacoronavirus 1'}}}}}}}}}}}}


In [ ]:
def double_deep(t0, t1, i0, i1):
    k0, v0 = t0.